## theory analysis 

First axiom: the parallelism number $t_i$ is the max CUDA thread number, the performance is the best for the function $g_i$. then we know the signature process including the public key generation, signing, and verifying. Those processes need serial run the function $g_i$. Not the optimal approach is use the parallelism number $t_i$ to the max CUDA thread number.
The performance is not the best for the function $g_i$.

Those have the Adaptive Thread function (AT) $AT:G\rightarrow T$, which maps each function $g_i \in G$ to its optimal thread count $t_i \in T$.


To accurately define this mapping, we can approach it through empirical performance modeling:

1. **Empirical Performance Model**: For each function $g_i$, we model execution time as:

   $T(g_i, t) = \alpha_i + \frac{\beta_i}{t} + \gamma_i \cdot t$

   Where:

   - $\alpha_i$ represents fixed overhead cost
   - $\frac{\beta_i}{t}$ captures the parallel speedup component
   - $\gamma_i \cdot t$ models thread management overhead
   - $t$ is the thread count

2. **Parameter Estimation**: We can estimate $\alpha_i$, $\beta_i$, and $\gamma_i$ by running benchmarks with varying thread counts and performing regression analysis.

3. **Optimal Thread Count**: The optimal thread count $t_i^*$ for function $g_i$ can be derived by finding the minimum of $T(g_i, t)$:

   $t_i^* = \sqrt{\frac{\beta_i}{\gamma_i}}$



4. **Implementation Strategy**: For practical implementation, we can:
   - Perform offline profiling for each key function
   - Build a lookup table mapping functions to their optimal thread counts
   - Apply dynamic thread adjustment based on runtime conditions

here we have one kernel function $g_i$, the blocks*threads is the $t_i$, the time is the $T(g_i, t)$, the per op is the $\frac{T(g_i, t)}{n}$, where $n$ is the number of operations of kernel function.

```csv
blocks, threads, time(ms), per op(ms)
32, 64, 310.03, 0.0095
64, 64, 155.41, 0.0047
96, 64, 118.89, 0.0036
128, 64, 90.56, 0.0028
160, 64, 102.16, 0.0031
192, 64, 85.41, 0.0026
224, 64, 87.52, 0.0027
```
